<a href="https://colab.research.google.com/github/RogerioPiazzon/trabalhofinalPECE/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install maritalk

In [ ]:
import json
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import numpy as np
import re,unicodedata,nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer, TFBertForSequenceClassification, TFXLMRobertaForSequenceClassification
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import keras
# import maritalkx


In [ ]:
class SingletonMeta(type):
    """
    The Singleton class can be implemented in different ways in Python. Some
    possible methods include: base class, decorator, metaclass. We will use the
    metaclass because it is best suited for this purpose.
    """

    _instances = {}

    def __call__(cls, *args, **kwargs):
        """
        Possible changes to the value of the `__init__` argument do not affect
        the returned instance.
        """
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

#INTENT RECOGINITION

In [ ]:
class UtilsData():
  def __init__(self):
    self.json_classes = r"/content/drive/MyDrive/PECE/resoucers/dict_classes.json"
    self.dict_classes = self.load_classes()
    self.num_classes = len(list(self.dict_classes.keys()))

  def load_classes(self):
      f = open(self.json_classes)
      dict = json.load(f)
      f.close()
      return dict

In [ ]:
class UtilsBert(UtilsData):
  def __init__(self):
    self.thresold_multilabel = 0.50
    super().__init__()
    pass

  def tokenization(self,sent):
    input_ids,attention_masks=list(),list()
    bert_inp=self.tokenizer.encode_plus(sent, add_special_tokens = True, max_length =64, pad_to_max_length = True, return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])
    input_ids=np.asarray(input_ids)
    attention_masks=np.array(attention_masks)
    return input_ids, attention_masks

  def identify_intention(self,sent,response="label"):
    input,attention = self.tokenization(sent)
    preds = self.model.predict([input,attention],batch_size=1, verbose=0)

    if response == "label":
      return self.label_classification(preds)
    else:
      return self.multilabel_classification(preds)

  def multilabel_classification(self,preds):
    labels = list()
    idx=np.where(preds.logits>=self.thresold_multilabel,1,0)
    for i,col in enumerate(idx[0]):
      if col == 1:
        labels.append((self.dict_classes[str(i)],preds.logits[0][i]))
    return labels

  def label_classification(self,preds):
    pred_labels = np.argmax(preds.logits, axis=1)
    return self.dict_classes[str(pred_labels[0])]

In [ ]:
class FineTuningBertimbau(UtilsBert,metaclass=SingletonMeta):
  def __init__(self):
    super().__init__()

    self.model_id = 'neuralmind/bert-base-portuguese-cased'
    self.weights = '/content/drive/MyDrive/PECE/models/bertimbau_model.h5'

    self.tokenizer =  AutoTokenizer.from_pretrained(self.model_id)
    self.model = TFBertForSequenceClassification.from_pretrained(self.model_id,num_labels=self.num_classes)
    self.model.load_weights(self.weights)

In [ ]:
class FineTuningRoberta(UtilsBert,metaclass=SingletonMeta):
  def __init__(self):
    super().__init__()

    self.model_id = "xlm-roberta-base"
    self.weights = '/content/drive/MyDrive/PECE/models/roberta_model.h5'

    self.tokenizer =  AutoTokenizer.from_pretrained(self.model_id)
    self.model = TFXLMRobertaForSequenceClassification.from_pretrained(self.model_id,num_labels=self.num_classes)
    self.model.load_weights(self.weights)

In [ ]:
class FineTuningAlbertina(UtilsBert,metaclass=SingletonMeta):
  def __init__(self):
    super().__init__()

    self.model_id = "xlm-roberta-base"
    self.weights = '/content/drive/MyDrive/PECE/models/albertina_model.h5'

    self.tokenizer =  AutoTokenizer.from_pretrained(self.model_id)
    self.model = TFXLMRobertaForSequenceClassification.from_pretrained(self.model_id,num_labels=self.num_classes)
    self.model.load_weights(self.weights)

#AGENTS TEXT-TO-SQL
Avaliar uso do langchain

In [ ]:
class AgentMaritaca():
  def __init__(self):

    self.API_KEY = "115462282570231901569$d61cc7ec5dc7a746133c922b77e5469a632ec6b89d690b81d5d0058684ba6dd0"
    self.model = maritalk.MariTalk(key=self.API_KEY)

    def generate_prompt (self,instruction, input=None,descritivo=None):
      return f""" Escreva uma consulta SQL para o SQLITE que atenda a seguinte afirmacao: {instruction}.
    Contexto:
    {input}

    Consulta SQL:"""

    def generate(self):
      answer = self.model.generate(
          generate_prompt("Quantos funcionarios ativos eu tenho atualmente?",description),
          chat_mode=True,
          do_sample=False,
          max_tokens=20
      )

    return answer.strip()






In [ ]:
!pip install -U -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.4 MB/s eta 0:00:00


In [ ]:
!pip install -q bitsandbytes accelerate xformers einops langchain faiss-cpu transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 M

In [ ]:
from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
NVIDIA A100-SXM4-40GB


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-v0.1"

bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
from accelerate import Accelerator
save_directory="/content/drive/MyDrive/PECE/models/sharded-mixtral"
accelerator = Accelerator()

# Here i am taking a Flan T5 XL fine tuned model (around 10GB) and creating shards of 2 GB each
accelerator.save_model(model=model, save_directory=save_directory,max_shard_size="2GB")

In [ ]:



text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

text = "O que é o Mistral?"
mistral_llm.invoke(text)


'\n\nO mistral (do provençal *mistrau*, "maestrale") é um vento frio e seco, de direção noroeste a nordeste, que sopra no sul da França. É mais intenso na região do Ródano-Alpes, onde pode atingir velocidades superiores a 100 km/h. A sua origem está relacionada com as diferenças de pressão'

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your
fantasy football knowledge. Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
class AgentMistral():
  def __init__(self):

    pass

#USO

In [ ]:
question = "Ocupacao do lucas piazzon"

In [ ]:
bertimbau = FineTuningBertimbau()

print(bertimbau.identify_intention(question))
print(bertimbau.identify_intention(question,response="multilabel"))

CARGO_DO_FUNCIONARIO
[('CARGO_DO_FUNCIONARIO', 7.0196404), ('DEPARTAMENTO_DO_FUNCIONARIO', 1.0808362), ('METRICAS_DE_CUSTO_DE_PRODUTOS', 2.1791675), ('METRICAS_DE_PRODUCAO_DE_PRODUTOS', 0.6497695)]
